### Setup

In [1]:
from IPython.display import clear_output
!pip install transformers
clear_output()

### Import Libraries

In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [3]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

### Scraping blog post

In [5]:
URL = "https://medium.com/flutter/whats-new-in-flutter-3-3-893c7b9af1ff"

In [6]:
r = requests.get(URL)

In [8]:
r.text

'<!doctype html><html lang="en"><head><title data-rh="true">What’s new in Flutter 3.3. Exciting updates for text handling… | by Kevin Jamaul Chisholm | Flutter | Aug, 2022 | Medium</title><meta data-rh="true" charset="utf-8"/><meta data-rh="true" name="viewport" content="width=device-width,minimum-scale=1,initial-scale=1,maximum-scale=1"/><meta data-rh="true" name="theme-color" content="#000000"/><meta data-rh="true" name="twitter:app:name:iphone" content="Medium"/><meta data-rh="true" name="twitter:app:id:iphone" content="828256236"/><meta data-rh="true" property="al:ios:app_name" content="Medium"/><meta data-rh="true" property="al:ios:app_store_id" content="828256236"/><meta data-rh="true" property="al:android:package" content="com.medium.reader"/><meta data-rh="true" property="fb:app_id" content="542599432471018"/><meta data-rh="true" property="og:site_name" content="Medium"/><meta data-rh="true" property="og:type" content="article"/><meta data-rh="true" property="article:published_

In [9]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [10]:
ARTICLE

"Flutter Aug 30 What’s new in Flutter 3.3 Exciting updates for text handling, performance improvements, and more! Welcome to Flutter 3.3! Only three months ago we announced Flutter 3, a massive milestone that included stable support for all platforms! The good news is that the momentum since this major release has not slowed down. Since the release of Flutter 3, Flutter has merged 5,687 pull requests. This release brings updates to Flutter web, desktop, text handling improvements in performance — and much more! We are also introducing exciting updates to the go_router package, DevTools, and VS Code extensions. Keep reading to learn more. Framework Until now, Flutter web apps didn’t match the expected behavior when attempting to select text. Like Flutter apps, native web applications are composed of a tree of elements. In a traditional web app, you can select multiple web elements with a single sweeping gesture, something that couldn’t be easily done on a Flutter web app. Today that all

### Chunk text

In [11]:
ARTICLE = ARTICLE.replace(".", ".<eos>")
ARTICLE = ARTICLE.replace("!", "!<eos>")
ARTICLE = ARTICLE.replace("?", "?<eos>")
sentences = ARTICLE.split("<eos>")

In [12]:
sentences

['Flutter Aug 30 What’s new in Flutter 3.',
 '3 Exciting updates for text handling, performance improvements, and more!',
 ' Welcome to Flutter 3.',
 '3!',
 ' Only three months ago we announced Flutter 3, a massive milestone that included stable support for all platforms!',
 ' The good news is that the momentum since this major release has not slowed down.',
 ' Since the release of Flutter 3, Flutter has merged 5,687 pull requests.',
 ' This release brings updates to Flutter web, desktop, text handling improvements in performance — and much more!',
 ' We are also introducing exciting updates to the go_router package, DevTools, and VS Code extensions.',
 ' Keep reading to learn more.',
 ' Framework Until now, Flutter web apps didn’t match the expected behavior when attempting to select text.',
 ' Like Flutter apps, native web applications are composed of a tree of elements.',
 ' In a traditional web app, you can select multiple web elements with a single sweeping gesture, something that

In [13]:
max_chunk = 500
current_chunk = 0
chunks = []

for sentence in sentences:
    if len(chunks) == current_chunk +1:
        if len(chunks[current_chunk]) + len(sentence.split(" ")) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(" "))
        else:
            current_chunk += 1
            chunks.append(sentence.split(" "))
    else:
        print(current_chunk)
        chunks.append(sentence.split(" "))

0


In [15]:
chunks[0]

['Flutter',
 'Aug',
 '30',
 'What’s',
 'new',
 'in',
 'Flutter',
 '3.',
 '3',
 'Exciting',
 'updates',
 'for',
 'text',
 'handling,',
 'performance',
 'improvements,',
 'and',
 'more!',
 '',
 'Welcome',
 'to',
 'Flutter',
 '3.',
 '3!',
 '',
 'Only',
 'three',
 'months',
 'ago',
 'we',
 'announced',
 'Flutter',
 '3,',
 'a',
 'massive',
 'milestone',
 'that',
 'included',
 'stable',
 'support',
 'for',
 'all',
 'platforms!',
 '',
 'The',
 'good',
 'news',
 'is',
 'that',
 'the',
 'momentum',
 'since',
 'this',
 'major',
 'release',
 'has',
 'not',
 'slowed',
 'down.',
 '',
 'Since',
 'the',
 'release',
 'of',
 'Flutter',
 '3,',
 'Flutter',
 'has',
 'merged',
 '5,687',
 'pull',
 'requests.',
 '',
 'This',
 'release',
 'brings',
 'updates',
 'to',
 'Flutter',
 'web,',
 'desktop,',
 'text',
 'handling',
 'improvements',
 'in',
 'performance',
 '—',
 'and',
 'much',
 'more!',
 '',
 'We',
 'are',
 'also',
 'introducing',
 'exciting',
 'updates',
 'to',
 'the',
 'go_router',
 'package,',
 'Dev

In [16]:
for chunk_id in range(len(chunks)):
    chunks[chunk_id] = " ".join(chunks[chunk_id])

In [18]:
chunks[0]

'Flutter Aug 30 What’s new in Flutter 3. 3 Exciting updates for text handling, performance improvements, and more!  Welcome to Flutter 3. 3!  Only three months ago we announced Flutter 3, a massive milestone that included stable support for all platforms!  The good news is that the momentum since this major release has not slowed down.  Since the release of Flutter 3, Flutter has merged 5,687 pull requests.  This release brings updates to Flutter web, desktop, text handling improvements in performance — and much more!  We are also introducing exciting updates to the go_router package, DevTools, and VS Code extensions.  Keep reading to learn more.  Framework Until now, Flutter web apps didn’t match the expected behavior when attempting to select text.  Like Flutter apps, native web applications are composed of a tree of elements.  In a traditional web app, you can select multiple web elements with a single sweeping gesture, something that couldn’t be easily done on a Flutter web app.  T

In [20]:
len(chunks[0].split(" "))

483

### Summarization

In [21]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [23]:
len(chunks)

4

In [24]:
len(res)

4

In [25]:
res

[{'summary_text': ' Flutter 3. 3 brings updates to Flutter web, desktop, text handling improvements, performance improvements, and much more . We are also introducing exciting updates to the go_router package, DevTools, and VS Code extensions .'},
 {'summary_text': ' Windows desktop application versions can now be set from your projects pubspec. yaml file and build arguments . This makes it easier to enable auto updates for your end customers to get the latest and greatest when an application update is pushed . The go router package, maintained by the Flutter team, simplifies routing by providing a declarative, URL-based API .'},
 {'summary_text': ' In the 2.10 stable release, we enabled Dart’s pointer compression optimization on iOS . The optimization reduces the amount of memory available for use by other components that reserve their own memory, for example, Flutter plugins . Fragment shaders written in GLSL and listed under a shaders: section in the Flutter manifest .'},
 {'summary

In [26]:
" ".join([summary["summary_text"] for summary in res])

" Flutter 3. 3 brings updates to Flutter web, desktop, text handling improvements, performance improvements, and much more . We are also introducing exciting updates to the go_router package, DevTools, and VS Code extensions .  Windows desktop application versions can now be set from your projects pubspec. yaml file and build arguments . This makes it easier to enable auto updates for your end customers to get the latest and greatest when an application update is pushed . The go router package, maintained by the Flutter team, simplifies routing by providing a declarative, URL-based API .  In the 2.10 stable release, we enabled Dart’s pointer compression optimization on iOS . The optimization reduces the amount of memory available for use by other components that reserve their own memory, for example, Flutter plugins . Fragment shaders written in GLSL and listed under a shaders: section in the Flutter manifest .  Flutter is Google's mobile UI framework for crafting high-quality native i

### Save as a text file

In [27]:
text = " ".join([summary["summary_text"] for summary in res])

In [29]:
with open("flutter3_summary.txt", "w") as f:
    f.write(text)